In [2]:

from dotenv import load_dotenv
import duckdb
from pathlib import Path

# testing
load_dotenv()

True

In [7]:
base_path = Path().cwd().parent
source_path = base_path / Path('silver/anilist/tag/tag-2024-1-15.parquet')
target_path = base_path / Path('gold/anilist/dim-tag.parquet')

In [8]:
tb = duckdb.read_parquet(str(source_path))

In [9]:
tb.limit(10).pl()

id,name,anime_id
i32,str,i32
98,"""""Female Protag…",7
40,"""""Police""""",7
29,"""""Magic""""",7
321,"""""Urban Fantasy…",7
456,"""""Conspiracy""""",7
327,"""""Noir""""",7
179,"""""Witch""""",7
226,"""""Fugitive""""",7
109,"""""Primarily Adu…",7


In [11]:
tb_dim = duckdb.sql("""
WITH tb_filter_col AS (
    SELECT
        id AS tag_id
        , name
    FROM tb
), dedupe AS (
    SELECT
        tag_id
        , ANY_VALUE(name) AS name
    FROM tb_filter_col
    GROUP BY (tag_id )
)
SELECT *
FROM dedupe
""")

In [12]:
tb_dim.shape

(287, 2)

In [13]:
tb_dim.to_parquet(str(target_path))